# Inleiding

### Use case
Titanic algemeen, kans op overleven, maar zou jij het overleefd hebben?!
Met Machine Learning kun je middels algoritmes bepalen of iemand wel of niet overleefd heeft gezien de beschikbare variabelen.  

### Instructies omgeving
Stukje over Google Colab met de belangrijkste toets-combinaties / ... 

### Imports & settings
Importeer de functies benodigd voor de training (of... dit gebeurd in cellen vooraf door cursusleider)

In [ ]:
# Imports
from scripts.preprocess.preprocessing import return_config, laden_data, give_input 
# from scripts.evaluation.plotting import 

# Settings

Toelichting Data Science, train/test principe. 

Laten we eens gaan kijken naar de dataset voor het trainen en testen van een model...


In [ ]:
df_train, df_test = laden_data()
display(df_train.head(), df_test.head())

Toelichting dataset (naslagwerk):
...

Laten we onszelf toevoegen aan de dataset.

Mogelijke vraag (om te kijken of ze opletten/begrepen hebben): Welke dataset zou dat zijn?!


In [ ]:
give_input()

In [ ]:
x = input("Geef str:")

toevoegen_passagiers() # pop-up venster?

In [ ]:
x

EDA

Toelichting trainen / algoritme?
Iets over classificatie algoritme (x aantal opties). Nu zijn immers 2 opties: Wel / niet overleven. 

Willen we nog iets doen met train_test_split? Zo ja, dan test set hernoemen in tekst?

Trainen van model / evaluatie?

Voorspellen testset met deelnemers training + visualisaties